<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/M6_threads_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo threads en Python

In [ ]:
%%writefile threads.py
import random
import sys
import threading
import os

def generate_random_matrix(N, min_value, max_value):
    matrix = []
    for i in range(N):
        row = []
        for j in range(N):
            row.append(random.randint(min_value, max_value))
        matrix.append(row)
    return matrix

def matrix_multiplication(matrixA, matrixB):
    result = []
    for i in range(len(matrixA)):
        row = []
        for j in range(len(matrixB[0])):
            sum = 0
            for k in range(len(matrixB)):
                sum += matrixA[i][k] * matrixB[k][j]
            row.append(sum)
        result.append(row)
    return result

def concurrent_matrix_multiplication(matrixA, matrixB):
    result = []
    threads = []
    for i in range(len(matrixA)):
        row = []
        # row va a contener la fila i de la matriz resultante. La inicializamos con ceros
        for j in range(len(matrixB[0])):
            row.append(0)
        result.append(row)

        # Creamos tantos hilos como filas tenga la matriz A. Estos hilos calcularan el producto entre la fila i de la matriz A y la matriz B
        t = threading.Thread(target=calculate_row, args=(matrixA, matrixB, result, i))
        t.start()
        threads.append(t)
    for t in threads:
        # Esperamos a los hilos que todavia esten calculando antes de devolver el resultado
        t.join()
    return result

def calculate_row(matrixA, matrixB, result, row_index):
    for j in range(len(matrixB[0])):
        sum = 0
        for k in range(len(matrixB)):
            sum += matrixA[row_index][k] * matrixB[k][j]
        result[row_index][j] = sum

def print_matrix(matrix):
    for row in matrix:
        print(row)

def parse_parameters():
    if len(sys.argv) != 2:
        print(f"Uso: python {os.path.basename(__file__)} N")
        sys.exit(1)
    N = int(sys.argv[1])
    if N < 5 or N > 20:
        print("N debe estar entre 5 y 20")
        sys.exit(1)
    return N

MIN_VALUE = -32
MAX_VALUE = 32
if __name__ == "__main__":
    N = parse_parameters()

    matrixA = generate_random_matrix(N, MIN_VALUE, MAX_VALUE)
    matrixB = generate_random_matrix(N, MIN_VALUE, MAX_VALUE)

    sequential_result = matrix_multiplication(matrixA, matrixB)
    concurrent_result = concurrent_matrix_multiplication(matrixA, matrixB)

    if sequential_result == concurrent_result:
        print("Matriz A:")
        print_matrix(matrixA)
        print("Matriz B:")
        print_matrix(matrixB)
        print("Resultado secuencial:")
        print_matrix(sequential_result)
        print("Resultado concurrente:")
        print_matrix(concurrent_result)
    else:
        print("Las dos matrices no dieron igual!!")

Writing threads.py


In [ ]:
!python threads.py 10

Matriz A:
[-14, -12, -13, 22, 17, 30, 4, -22, -13, -29]
[-20, 28, 28, 28, 15, -30, 15, -29, -8, 32]
[29, -30, 19, -13, -24, 12, 22, 19, 26, 26]
[-29, -2, -14, -30, 8, -7, -30, 2, 22, 7]
[8, -17, 17, 27, 24, -8, 9, -6, -6, -10]
[-7, 26, 30, -8, 26, 24, -8, 17, -1, 6]
[-2, -28, 14, -23, 14, -3, -14, -11, 23, -2]
[30, 22, 0, 31, -26, 20, 2, 13, -19, 25]
[6, -26, 1, -18, 7, 25, 21, 9, 21, 24]
[11, 16, 15, -3, 10, -28, -27, 25, -20, 20]
Matriz B:
[-3, 19, 31, 9, -32, -14, 5, -3, 17, 12]
[21, -21, 20, -13, -27, -17, -5, 22, 14, -24]
[21, 23, 26, 16, -30, 8, 4, -26, 11, -16]
[3, 0, 5, -11, -25, 5, 6, 31, 2, 9]
[-18, -17, -7, -10, 3, -2, 32, 23, 15, 31]
[-14, -5, 7, -27, 29, 24, 16, -27, 25, -19]
[-21, 9, -11, 24, 16, -6, 0, -32, 25, 32]
[5, 7, -9, -1, 5, 17, -2, 17, -24, -9]
[30, -18, -1, -31, 30, -6, 23, -27, 7, -13]
[-18, -22, -20, -9, 23, 28, -30, -2, -5, -18]
Resultado secuencial:
[-1205, 2, -64, -618, 430, -40, 1709, 286, 1182, 1500]
[194, -1057, -43, 605, -1890, -221, -1046, 1150, 746, 